In [237]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from dateutil import parser
from datetime import datetime
import re
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import metrics
from sklearn.metrics import classification_report

In [249]:
 data = pd.read_csv("Supp Prediction.csv")

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_22172\3065241903.py:1: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Supp Prediction.csv")


In [ ]:
"claim_status","exposure_status","primary_point_of_impact","loss_cause","company","line_of_business","loss_party_type","policy_type","coverage_grouping"

In [295]:
def dict_maker1(df,columns):
    dict_values = []
    for column in columns:
        keys = list(set(df[column]))
        values = list(range(len(set(df[column]))))
        res = {keys[i]: values[i] for i in range(len(keys))}
        dict_values.append(res)
    dic = {columns[i]: dict_values[i] for i in range(len(columns))}
    return dic

In [296]:
dict_maker1(data,["claim_status","exposure_status","primary_point_of_impact","loss_cause","company","line_of_business","loss_party_type","policy_type","coverage_grouping"])

{'claim_status': {'Open': 0, 'Closed': 1},
 'exposure_status': {'Open': 0, 'Closed': 1},
 'primary_point_of_impact': {nan: 0,
  'Hood': 1,
  'Left Rear Door': 2,
  'Deck Lid': 3,
  'Center Rear': 4,
  'Right Front Door': 5,
  'Right Front Corner': 6,
  'Roof Rear': 7,
  'Roof Front': 8,
  'Left Front Door': 9,
  'Left Rear Side': 10,
  'Left Front Side': 11,
  'Right Rear Side': 12,
  'Left Front Corner': 13,
  'Left Rear Corner': 14,
  'Right Rear Door': 15,
  'Right Rear Corner': 16,
  'Right Front Side': 17,
  'Center Front': 18},
 'loss_cause': {'Falling, Flying, or Moving Object': 0,
  'Other': 1,
  'Fire': 2,
  'Earthquake/Earth Movement': 3,
  'Theft': 4,
  'Vandalism/Malicious Mischief': 5,
  'Emergency Roadside Service': 6,
  'Glass Breakage': 7,
  'Collision with Motorcycle': 8,
  'Collision with Pedestrian/Bicyclist': 9,
  'Collision with Train/Bus': 10,
  'Hail': 11,
  'Loading/Unloading Damage': 12,
  'Animal': 13,
  'Collision with Roadbed': 14,
  'Collision with Motor Ve

In [340]:
def prep_df(csv):
    global df_original
    df_original = pd.read_csv(csv)
    df = pd.read_csv(csv)
    df["date_today"] = datetime.now()
    df = df.dropna(subset = ["original_estimate_complete_datetime"])
    df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
    df = df.assign(time_since_estimate =  df['date_today'] - df['estimate_date_parse'])
    df["days_since_estimate"] = df["time_since_estimate"].apply(lambda x: int(str(x).split()[0]))
    del df["time_since_estimate"]
    #df = df[df["days_since_estimate"] >= 90]
    df1 = df[df["suppCnt"] == 0]
    df2 = df[df["suppCnt"] != 0]
    df1["suppInd"] = 0
    df2["suppInd"] = 1
    df = pd.concat([df1,df2], axis = 0)
    df["primary_point_of_impact"].fillna(0,inplace = True)
    df["total_loss_points"].fillna(999,inplace = True)
    df["insured_fault_pct"].fillna(999,inplace = True)
    #df = df.dropna(subset = ["insured_fault_pct"])
    global cleanup_nums
    cleanup_nums = dict_maker1(data,["claim_status","exposure_status","primary_point_of_impact","loss_cause","company","line_of_business","loss_party_type","policy_type","coverage_grouping"])   
                   
    df = df.replace(cleanup_nums)
    df["paid_loss_amount"] = df["paid_loss_amount"].apply(lambda x: abs(x))
    return df

In [341]:
df = prep_df("Supp Prediction.csv")

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_22172\3890678509.py:3: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_22172\3890678509.py:4: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_22172\3890678509.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_22172\3890678509.py:14: SettingWithCopyWarning: 
A value is trying to be set on 

In [371]:
len(df[df["suppInd"] == 1][df["days_since_estimate"]>= 90]) / len(df[df["days_since_estimate"]>= 90])

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_22172\1371189825.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(df[df["suppInd"] == 1][df["days_since_estimate"]>= 90]) / len(df[df["days_since_estimate"]>= 90])


0.5407823198005587

In [372]:
df[df["suppInd"] == 1][df["days_since_estimate"]>= 90]

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_22172\2120652097.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df["suppInd"] == 1][df["days_since_estimate"]>= 90]


,airbags_deployed_ind,all_over_damage_ind,claim_number,claim_status,company,coverage_grouping,detailed_loss_cause,driveable_ind,early_tow_ind,exposure_id,...,vehicle_condition_type,vehicle_make,vehicle_model,vehicle_year,vin,zeroRowValidation,date_today,estimate_date_parse,days_since_estimate,suppInd
5,1.0,0,01-004-319140,1,1,0,Hit Deer,No,NaN,11499422,...,NaN,Nissan,Murano 4d 4wd,2021,5N1AZ2DS5MC118045,0,2022-08-05 10:46:14.241579,2022-01-24 15:00:43,192,1
20,1.0,0,01-004-034939,1,1,1,Collision While Turning,No,1.0,10922444,...,NaN,Toyota,Camry Hybrid 4d 2wd,2021,4T1S31AK4MU564671,0,2022-08-05 10:46:14.241579,2021-11-12 14:19:46,265,1
22,1.0,0,01-003-596475,1,1,1,T-bone,No,NaN,9826261,...,NaN,Honda,Pilot 4d 4wd,2014,5FNYF4H5XEB041513,0,2022-08-05 10:46:14.241579,2021-07-13 18:08:03,387,1
36,1.0,0,01-004-267181,1,1,0,Curb,No,NaN,12020448,...,NaN,Mazda,Cx-5 4d 4wd,2017,JM3KFBCLXH0226848,0,2022-08-05 10:46:14.241579,2022-01-18 13:43:05,198,1
40,1.0,0,01-004-430123,1,1,1,Insured Rear-ended Claimant,Yes,NaN,11928279,...,NaN,Nissan,Rogue 4d 4wd,2010,JN8AS5MV3AW148938,0,2022-08-05 10:46:14.241579,2022-03-07 15:01:35,150,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286606,NaN,0,01-003-997447,1,1,0,Hit Deer,Yes,NaN,10845627,...,NaN,Ford Truck,Edge 4d 2wd,2013,2FMDK3JC5DBE36235,0,2022-08-05 10:46:14.241579,2021-10-28 13:36:28,280,1
286609,NaN,0,01-001-618495,1,1,1,Hit and Run,Yes,NaN,3745372,...,NaN,Toyota,Corolla Sedan 2wd,2019,2T1BURHE9KC169296,0,2022-08-05 10:46:14.241579,2022-01-03 08:42:20,214,1
286613,NaN,0,01-002-469312,1,1,1,Sideswipe,No,NaN,5404973,...,NaN,Hyundai,Elantra 4d,2017,5NPD84LF4HH055115,0,2022-08-05 10:46:14.241579,2020-07-20 14:23:19,745,1
286615,NaN,0,01-002-491095,1,1,1,Insured Rear-ended Claimant,Yes,NaN,5444320,...,NaN,Suzuki,Grand Vitara 4d 4x2,2008,JS3TE941X84102370,0,2022-08-05 10:46:14.241579,2020-06-30 10:39:52,766,1


In [343]:
train, test = train_test_split(df, random_state=0, train_size = .75)

In [344]:
X = df[["insured_fault_pct","days_since_estimate","vehicle_year","claim_status","exposure_status","primary_point_of_impact","loss_cause","company","line_of_business","loss_party_type","policy_type","coverage_grouping"]]
Y = df[["payAfterCloseInd"]]

In [345]:
best_features = SelectKBest(score_func=chi2, k=3)
fit= best_features.fit(X,Y)

In [346]:
df_scores= pd.DataFrame(fit.scores_)
df_columns= pd.DataFrame(X.columns)
features_scores= pd.concat([df_columns, df_scores], axis=1)
features_scores.columns= ['Features', 'Score']
features_scores.sort_values(by = 'Score')

,Features,Score
8,line_of_business,0.792856
2,vehicle_year,2.014975
10,policy_type,3.565951
5,primary_point_of_impact,52.984171
7,company,99.551232
4,exposure_status,163.201420
9,loss_party_type,304.357983
6,loss_cause,1177.493674
11,coverage_grouping,1354.318694
3,claim_status,2531.658540


In [347]:
X= df[['loss_cause', 'primary_point_of_impact', 'insured_fault_pct']]
Y= df[['claim_number','payAfterCloseInd']]
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.4,random_state=100)
logreg= LogisticRegression()
logreg.fit(X_train,y_train[['payAfterCloseInd']])
y_pred=logreg.predict(X_test)
print (X_test)
print (y_pred)

C:\Users\TWILLI16\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


        loss_cause  primary_point_of_impact  insured_fault_pct
244296          13                       18                0.0
268581          15                       11                0.0
230018          15                       14              100.0
66584           11                        1                0.0
62812           13                       13                0.0
...            ...                      ...                ...
249900          15                       18              100.0
233210          15                       17              100.0
258969           1                        0                0.0
161623          16                       13                0.0
80696            0                        2                0.0

[114502 rows x 3 columns]
[0 0 0 ... 0 0 0]


In [348]:
y_test["prediction"] = y_pred

In [349]:
len(y_test[y_test["payAfterCloseInd"]==y_test["prediction"]]) / len(y_test)

0.833871897434106

In [362]:
set(y_test["prediction"])

{0}

In [373]:
X_last90 = df[df["days_since_estimate"] <= 90]
X_last90 = X_last90[['loss_cause', 'primary_point_of_impact', 'insured_fault_pct']]
X_over90 = df[df["days_since_estimate"] > 90]
X_over90 = X_over90[['loss_cause', 'primary_point_of_impact', 'insured_fault_pct']]
Y_last90 = df[df["days_since_estimate"] <= 90]
Y_last90 = Y_last90[['claim_number','suppInd']]
Y_over90 = df[df["days_since_estimate"] > 90]
Y_over90 = Y_over90[['claim_number','suppInd']]
#X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.4,random_state=100)
logreg= LinearRegression()
logreg.fit(X_over90,Y_over90[['suppInd']])
y_pred = logreg.predict(X_last90)
print (X_last90)
print (y_pred)

        loss_cause  primary_point_of_impact  insured_fault_pct
1               15                       18              100.0
6               15                       10                0.0
17              15                       11                0.0
19              14                        0              100.0
30              15                       11              100.0
...            ...                      ...                ...
286588          15                       10              100.0
286592           2                       18                0.0
286598          11                        0                0.0
286610          11                        0                0.0
286612          11                        0                0.0

[94271 rows x 3 columns]
[[0.55725642]
 [0.55283471]
 [0.55498205]
 ...
 [0.51023687]
 [0.51023687]
 [0.51023687]]


In [374]:
Y_last90["prediction"] = y_pred
len(Y_last90[Y_last90["prediction"] == 1]) / len(Y_last90)
Y_last90

,claim_number,suppInd,prediction
1,01-004-717683,0,0.557256
6,01-004-683118,0,0.552835
17,01-004-715340,0,0.554982
19,01-004-992800,0,0.513323
30,01-004-609862,0,0.542225
...,...,...,...
286588,01-004-788524,1,0.540078
286592,01-004-818419,1,0.501359
286598,01-004-746012,1,0.510237
286610,01-004-720923,1,0.510237


In [360]:
X_last90

,loss_cause,primary_point_of_impact,insured_fault_pct
1,15,18,100.0
6,15,10,0.0
17,15,11,0.0
19,14,0,100.0
30,15,11,100.0
...,...,...,...
286588,15,10,100.0
286592,2,18,0.0
286598,11,0,0.0
286610,11,0,0.0


In [361]:
X_over90

,loss_cause,primary_point_of_impact,insured_fault_pct
0,15,6,0.0
2,15,18,100.0
3,15,6,100.0
4,15,18,100.0
7,15,17,100.0
...,...,...,...
286606,13,18,0.0
286609,16,16,0.0
286613,15,10,100.0
286615,15,18,100.0


In [365]:
len(data[data["payAfterCloseInd"] == 0][]) / len(data)

0.8332007563863711